# NBA Game Dashboard Design Outline

# Game Context

- away @ home
- total / spread

# Team Context

- record / L10 record
- injuries
    - players returning from injuries 
    - starting lineups ?
    - highlight key scorers / defenders
- stats allowed (overall + dvp)
- pace(rank) ?

# Player Context

- PTS, REB, AST (+ projection)
    - FPS
    - 3s
        - REB/AST chance data

- hot / cold n-game streak
    - x out of last y
    - season hit %
    - visualize outcomes vs szn/rolling avg
        - visualize attempts/chances
- baseline deviation ?
- matchup
- stats without injured teammate
- home / away splits
- H2H performances

- Similar player recently recorded stats
- Recent scoring/assist performances vs game totals 

In [5]:
home_team = "New Orleans Pelicans"
away_team = "Los Angeles Lakers"
game_total = 235
home_spread = -6
away_spread = 6

print(f"{away_team} @ {home_team}")
print(f"Game total: {game_total}")
print(f"Spreads:\n  Away: {away_spread}\n  Home: {home_spread}")

Los Angeles Lakers @ New Orleans Pelicans
Game total: 235
Spreads:
  Away: 6
  Home: -6


In [6]:
# Connect to database
from sqlalchemy import create_engine
import pandas as pd

DB_USER = "admin"
DB_PASSWORD = "admin"
DB_HOST = "localhost"
DB_PORT = 5433
DB_NAME = "nba_db"

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

In [21]:
query = """
ALTER TABLE boxscores.player_boxscores_traditional_v3
ADD COLUMN IF NOT EXISTS clean_name TEXT;

UPDATE boxscores.player_boxscores_traditional_v3
SET clean_name = regexp_replace(
    lower(unaccent(CONCAT_WS(' ', first_name, family_name))),
    '[^a-z ]',
    '',
    'g'
);

SELECT 
    *,
    AVG(points) OVER (
        PARTITION BY player_id
        ORDER BY game_id
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) AS ppg,
    AVG(points) OVER(
        PARTITION BY player_id
        ORDER BY game_id
        ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING
    ) AS r10_ppg
FROM boxscores.player_boxscores_traditional_v3
WHERE clean_name = 'luka doncic'
"""

pd.read_sql(query, engine).head()

,game_id_api,team_id,team_city,team_name,team_tricode,team_slug,player_id,first_name,family_name,name_initial,...,turnovers,fouls_personal,points,plus_minus_points,game_id,minutes_decimal,player_name,clean_name,ppg,r10_ppg
0,0022400007,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,5,2,31,2.0,0022400007,41.083332,Luka Dončić,luka doncic,NaN,NaN
1,0022400024,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,3,0,26,28.0,0022400024,29.616667,Luka Dončić,luka doncic,31.000000,31.000000
2,0022400056,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,3,4,37,16.0,0022400056,40.466667,Luka Dončić,luka doncic,28.500000,28.500000
3,0022400074,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,4,3,28,6.0,0022400074,36.283333,Luka Dončić,luka doncic,31.333333,31.333333
4,0022400095,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,2,4,40,-12.0,0022400095,39.416668,Luka Dončić,luka doncic,30.500000,30.500000
